## Basic Structured Operations


### Step 1: Initialize PySpark Session


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, lit , avg
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Create a Spark session
spark = SparkSession.builder.appName("day2").getOrCreate()


23/08/31 08:19:41 WARN Utils: Your hostname, riyaz-Aspire-VX5-591G resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface enp3s0)
23/08/31 08:19:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 08:19:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2: Load the Dataset


In [2]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "./occupation.csv"  # Replace with the actual path
occupation = spark.read.csv(data_path, header=True, inferSchema=True)


In [3]:
occupation.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



### Problem 1: Selecting Specific Columns
Problem: Select the "user_id," "age," and "occupation" columns from the occupation DataFrame.

In [13]:
# Create a temporary view
occupation.createOrReplaceTempView("occupation_view")


In [5]:
# Run SQL queries
query = "SELECT user_id, age, occupation FROM occupation_view"
result = spark.sql(query)
result.show()


+-------+---+-------------+
|user_id|age|   occupation|
+-------+---+-------------+
|      1| 24|   technician|
|      2| 53|        other|
|      3| 23|       writer|
|      4| 24|   technician|
|      5| 33|        other|
|      6| 42|    executive|
|      7| 57|administrator|
|      8| 36|administrator|
|      9| 29|      student|
|     10| 53|       lawyer|
|     11| 39|        other|
|     12| 28|        other|
|     13| 47|     educator|
|     14| 45|    scientist|
|     15| 49|     educator|
|     16| 21|entertainment|
|     17| 30|   programmer|
|     18| 35|        other|
|     19| 40|    librarian|
|     20| 42|    homemaker|
+-------+---+-------------+
only showing top 20 rows



### Problem 2: Filtering Rows based on Condition
Problem: Find the users who are older than 30 years from the occupation DataFrame.

In [7]:
query = " SELECT * FROM occupation_view WHERE age > 30"
result = spark.sql(query)
result.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      2| 53|     F|        other|   94043|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemaker|   95660|
|     25| 39|     M|     engineer|   55107|
|     26| 49|     M|     engineer|   21044|
|     27| 40|     F|    librarian|   30030|
|     28| 32|     M|       writer|   55369|
|     29| 41|     M|   programmer|   94043|
|     34| 38|     F|administrator|   42141|
|     39| 41|     M|entertainmen

### Problem 3: Counting and Grouping
Problem: Count the number of users in each occupation from the occupation DataFrame.

In [13]:
query = "SELECT occupation, COUNT(*) AS user_count FROM occupation_view GROUP BY occupation"
result = spark.sql(query)
result.show()

+-------------+----------+
|   occupation|user_count|
+-------------+----------+
|    librarian|        51|
|      retired|        14|
|       lawyer|        12|
|         none|         9|
|       writer|        45|
|   programmer|        66|
|    marketing|        26|
|        other|       105|
|    executive|        32|
|    scientist|        31|
|      student|       196|
|     salesman|        12|
|       artist|        28|
|   technician|        27|
|administrator|        79|
|     engineer|        67|
|   healthcare|        16|
|     educator|        95|
|entertainment|        18|
|    homemaker|         7|
+-------------+----------+
only showing top 20 rows



### Problem 4: Adding a New Column
Problem: Add a new column "age_group" to the occupation DataFrame based on the age of the users. Divide users into age groups: "18-25", "26-35", "36-50", and "51+".

In [4]:
occupation_with_age_group = occupation.withColumn(
    "age_group",
    when(col("age").between(18, 25), "18-25")
    .when(col("age").between(26, 35), "26-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("51+")
)
occupation_with_age_group.show()

+-------+---+------+-------------+--------+---------+
|user_id|age|gender|   occupation|zip_code|age_group|
+-------+---+------+-------------+--------+---------+
|      1| 24|     M|   technician|   85711|    18-25|
|      2| 53|     F|        other|   94043|      51+|
|      3| 23|     M|       writer|   32067|    18-25|
|      4| 24|     M|   technician|   43537|    18-25|
|      5| 33|     F|        other|   15213|    26-35|
|      6| 42|     M|    executive|   98101|    36-50|
|      7| 57|     M|administrator|   91344|      51+|
|      8| 36|     M|administrator|   05201|    36-50|
|      9| 29|     M|      student|   01002|    26-35|
|     10| 53|     M|       lawyer|   90703|      51+|
|     11| 39|     F|        other|   30329|    36-50|
|     12| 28|     F|        other|   06405|    26-35|
|     13| 47|     M|     educator|   29206|    36-50|
|     14| 45|     M|    scientist|   55106|    36-50|
|     15| 49|     F|     educator|   97301|    36-50|
|     16| 21|     M|entertai

In [24]:
query = "UPDATE occupation_view\
    SET age_group =\
    CASE\
        WHEN age BETWEEN 18 AND 25 THEN '18-25'\
        WHEN age BETWEEN 26 AND 35 THEN '26-35'\
        WHEN age BETWEEN 36 AND 50 THEN '36-50'\
        ELSE '51+'\
    END\
"
result = spark.sql(query)
result.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `age_group` cannot be resolved. Did you mean one of the following? [`occupation_view`.`age`, `occupation_view`.`gender`, `occupation_view`.`user_id`, `occupation_view`.`occupation`, `occupation_view`.`zip_code`].; line 1 pos 30;
'UpdateTable [assignment('age_group, CASE WHEN ((age#18 >= cast(18 as int)) AND (age#18 <= cast(25 as int))) THEN 18-25 WHEN ((age#18 >= cast(26 as int)) AND (age#18 <= cast(35 as int))) THEN 26-35 WHEN ((age#18 >= cast(36 as int)) AND (age#18 <= cast(50 as int))) THEN 36-50 ELSE 51+ END)]
+- SubqueryAlias occupation_view
   +- View (`occupation_view`, [user_id#17,age#18,gender#19,occupation#20,zip_code#21])
      +- Relation [user_id#17,age#18,gender#19,occupation#20,zip_code#21] csv


### Problem 5: Creating DataFrames and Converting to Spark Types
Problem: Given the provided code snippet, create a DataFrame df using the given data and schema. The schema includes columns for firstname, middlename, lastname, id, gender, and salary. After creating the DataFrame, print its schema and display its content without truncation.

In [5]:
#Defining Schema for our dataframe

schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("id", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", FloatType(), True)
])


#Inserting data into the newly made schema

data = [
    ("James", None, "Smith", 36636, "M", 3000.0),
    ("Michael", "Rose", None, 40288, "M", 4000.0),
    ("Robert", None, "Williams", 42114, "M", 4000.0),
    ("Maria", "Anne", "Jones", 39192, "F", 4000.0),
    ("Jen", "Mary", "Brown", None, "F", -1.0)
]

# Create a DataFrame using the schema and data

df = spark.createDataFrame(data, schema)


# Show the schema

df.printSchema()

# Display the content without truncation

df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: float (nullable = true)



+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |null      |Smith   |36636|M     |3000.0|
|Michael  |Rose      |null    |40288|M     |4000.0|
|Robert   |null      |Williams|42114|M     |4000.0|
|Maria    |Anne      |Jones   |39192|F     |4000.0|
|Jen      |Mary      |Brown   |null |F     |-1.0  |
+---------+----------+--------+-----+------+------+



### Problem 6: Adding and Renaming Columns
Problem: Add a new column "gender" to the existing DataFrame and rename the "Age" column to "Years".

In [6]:
# Add a new column "gender" with a constant value
occupation_with_gender = occupation.withColumn("gender", lit("Unknown"))

# Rename the "age" column to "years"
occupation_final = occupation_with_gender.withColumnRenamed("age", "years")

# Show the schema and content
occupation_final.printSchema()

occupation_final.show(truncate=False)

root
 |-- user_id: integer (nullable = true)
 |-- years: integer (nullable = true)
 |-- gender: string (nullable = false)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)

+-------+-----+-------+-------------+--------+
|user_id|years|gender |occupation   |zip_code|
+-------+-----+-------+-------------+--------+
|1      |24   |Unknown|technician   |85711   |
|2      |53   |Unknown|other        |94043   |
|3      |23   |Unknown|writer       |32067   |
|4      |24   |Unknown|technician   |43537   |
|5      |33   |Unknown|other        |15213   |
|6      |42   |Unknown|executive    |98101   |
|7      |57   |Unknown|administrator|91344   |
|8      |36   |Unknown|administrator|05201   |
|9      |29   |Unknown|student      |01002   |
|10     |53   |Unknown|lawyer       |90703   |
|11     |39   |Unknown|other        |30329   |
|12     |28   |Unknown|other        |06405   |
|13     |47   |Unknown|educator     |29206   |
|14     |45   |Unknown|scientist    |55106 

### Problem 7: Filtering Rows and Sorting
Problem: Filter out users who are younger than 30 years and sort the DataFrame by age in descending order.

In [9]:
# Filter out users younger than 30 years
filtered_occupation = occupation_final.filter(occupation.age >= 30)

# Sort the DataFrame by age in descending order
sorted_occupation = filtered_occupation.orderBy(occupation.age.desc())

# Show the resulting DataFrame
sorted_occupation.show()

+-------+-----+-------+-------------+--------+
|user_id|years| gender|   occupation|zip_code|
+-------+-----+-------+-------------+--------+
|    481|   73|Unknown|      retired|   37771|
|    767|   70|Unknown|     engineer|   00000|
|    803|   70|Unknown|administrator|   78212|
|    860|   70|Unknown|      retired|   48322|
|    559|   69|Unknown|    executive|   10022|
|    585|   69|Unknown|    librarian|   98501|
|    349|   68|Unknown|      retired|   61455|
|    573|   68|Unknown|      retired|   48911|
|    211|   66|Unknown|     salesman|   32605|
|    318|   65|Unknown|      retired|   06518|
|    564|   65|Unknown|      retired|   94591|
|    651|   65|Unknown|      retired|   02903|
|    423|   64|Unknown|        other|   91606|
|    845|   64|Unknown|       doctor|   97405|
|    364|   63|Unknown|     engineer|   01810|
|    777|   63|Unknown|   programmer|   01810|
|    858|   63|Unknown|     educator|   09645|
|    266|   62|Unknown|administrator|   78756|
|    520|   6

### Problem 8: Repartitioning and Collecting Rows
Problem: Repartition the DataFrame into 2 partitions without shuffling the data, then collect and display all rows in the driver and print number of partitions

In [10]:
# Repartition the DataFrame into 2 partitions without shuffling
repartitioned_df = df.coalesce(2)

# Collect and display all rows in the driver
all_rows = repartitioned_df.collect()
for row in all_rows:
    print(row)

# Get the number of partitions
num_partitions = repartitioned_df.rdd.getNumPartitions()

Row(firstname='James', middlename=None, lastname='Smith', id=36636, gender='M', salary=3000.0)
Row(firstname='Michael', middlename='Rose', lastname=None, id=40288, gender='M', salary=4000.0)
Row(firstname='Robert', middlename=None, lastname='Williams', id=42114, gender='M', salary=4000.0)
Row(firstname='Maria', middlename='Anne', lastname='Jones', id=39192, gender='F', salary=4000.0)
Row(firstname='Jen', middlename='Mary', lastname='Brown', id=None, gender='F', salary=-1.0)


In [11]:
print("Number of partitions:", num_partitions)

Number of partitions: 2


### Additional questions:

Use both spark SQL and Pyspark to obtain answer wherever relevant

#### Filter out rows where the age is greater than 30 and create a new DataFrame. Then, add a new column named "is_elderly" with a value of "True" for these rows and "False" otherwise.Rename the "gender" column to "sex".

In [14]:
# Spark SQL

# Create a temporary view to use Spark SQL
sorted_occupation.createOrReplaceTempView("occupation_view")

print(sorted_occupation)
## Filter out rows where age is greater than 30
filtered_rows_sql = spark.sql("SELECT user_id, age, gender AS sex, occupation, zip_code FROM occupation_view WHERE age > 30")

# Add a new column "is_elderly" with a value of True or False
filtered_rows_sql = filtered_rows_sql.withColumn("is_elderly", lit(True))

# Show the resulting DataFrame without truncation
filtered_rows_sql.show(100, False)


# Show the resulting DataFrame
filtered_rows_sql.show()

DataFrame[user_id: int, years: int, gender: string, occupation: string, zip_code: string]


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `age` cannot be resolved. Did you mean one of the following? [`occupation_view`.`gender`, `occupation_view`.`years`, `occupation_view`.`user_id`, `occupation_view`.`zip_code`, `occupation_view`.`occupation`].; line 1 pos 84;
'Project ['user_id, 'age, 'gender AS sex#215, 'occupation, 'zip_code]
+- 'Filter ('age > 30)
   +- SubqueryAlias occupation_view
      +- View (`occupation_view`, [user_id#17,years#108,gender#102,occupation#20,zip_code#21])
         +- Project [user_id#17, years#108, gender#102, occupation#20, zip_code#21]
            +- Sort [age#18 DESC NULLS LAST], true
               +- Project [user_id#17, years#108, gender#102, occupation#20, zip_code#21, age#18]
                  +- Filter (age#18 >= 30)
                     +- Project [user_id#17, age#18 AS years#108, gender#102, occupation#20, zip_code#21, age#18]
                        +- Project [user_id#17, age#18, Unknown A

In [ ]:
# Pyspark



#### Calculate the average age of male and female users separately. Present the result in a new DataFrame with columns "gender" and "avg_age".

In [ ]:
# Spark SQL



In [ ]:
# Pyspark



#### Add a new column named "full_name" to the dataset by concatenating the "user_id" and "occupation" columns. Then, rename the "zip_code" column to "postal_code" in the same DataFrame.

In [ ]:
# Spark SQL



In [ ]:
# Pyspark



#### Filter out rows where occupation is 'technician', select only the "user_id" and "age" columns, and then add a new column "age_diff" that calculates the difference between the user's age and the average age in the dataset.

In [ ]:
# Spark SQL



In [ ]:
# Pyspark



#### Divide the dataset into two DataFrames: one with male users and another with female users. Repartition both DataFrames to have 2 partitions each. Then, union these two DataFrames together and display the resulting DataFrame.

#### Create and fill a new DataFrame named user_ratings with columns user_id and rating max 10 column. Both user_data and user_ratings share the user_id column. Combine these two DataFrames to create a new DataFrame that includes user information and their corresponding ratings. Make sure to keep only the users present in both DataFrames.